<a href="https://colab.research.google.com/github/TechnoPolizzz/safety_doors/blob/main/SafetyDoors_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install open3d

In [3]:
import open3d as o3d
import cv2
import numpy as np

In [4]:
from google.colab.patches import cv2_imshow # Позволяет выводить изображения

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go

### Вспомогательные функции

In [6]:
def joinPath(path_name):
  return os.path.join(path_name["path"], path_name["name"])

In [7]:
# Объединение списков без повторений
def mergeLists(list1, list2):
  return list(set(list1 + list2))

### Разархивируем датасеты

In [ ]:
!unzip /content/drive/MyDrive/technoPolizzz/point_cloud_train.zip

In [ ]:
!mkdir -p pcd/clouds_tof
!unzip "point_cloud_train/clouds_tof/*.zip" -d pcd/clouds_tof

In [ ]:
!mkdir -p pcd/clouds_stereo
!unzip "point_cloud_train/clouds_stereo/*.zip" -d pcd/clouds_stereo

### Функции для визуализации

In [13]:
# Создаем mesh из обрамляющего прямоугольника
def createMeshFromBB(bb):
  bbcloud = o3d.geometry.PointCloud()
  bbcloud.points = bb.get_box_points()
  bb.color = bb.color
  hull, _= bbcloud.compute_convex_hull()
  return hull

In [14]:
def createMeshData(mesh, opacity=0.2):
  verts = np.asarray(mesh.vertices)
  triangs = np.asarray(mesh.triangles)

  mdata = go.Mesh3d(
        x=verts[:,0],
        y=verts[:,1],
        z=verts[:,2],
        i = triangs[:,0],
        j = triangs[:,1],
        k = triangs[:,2],
        opacity=opacity,
    )
  return mdata

In [15]:
def createScatter3dData(pcd):
  points = np.asarray(pcd.points)
  colors = np.asarray(pcd.colors)
  scdata = go.Scatter3d(
            x=points[:,0], 
            y=points[:,1], 
            z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors))
  return scdata

In [16]:
def drawGeometry(geometry, width=800, height=600, title=""):
  mydata = []
  for g in geometry:
    if type(g) is o3d.geometry.PointCloud:
      mydata.append(createScatter3dData(g))
    elif type(g) is o3d.geometry.AxisAlignedBoundingBox or type(g) is o3d.geometry.OrientedBoundingBox:
      m = createMeshFromBB(g)
      mydata.append(createMeshData(m))
    elif type(g) is o3d.geometry.TriangleMesh:
      mydata.append(createMeshData(g))
  
  fig = go.Figure(
    data=mydata,
    layout=dict(
        width = width,
        height = height,
        scene=dict(
            xaxis=dict(visible=True),
            yaxis=dict(visible=True),
            zaxis=dict(visible=True)
        ),
        title=title,
    )
  )
  fig.show()

In [17]:
def display_inlier_outlier(cloud, ind, in_color=None, out_color=[1, 0, 0], title=""):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    if not out_color is None:
      outlier_cloud.paint_uniform_color(out_color)
    if not in_color is None:
      inlier_cloud.paint_uniform_color(in_color)
    drawGeometry([inlier_cloud, outlier_cloud], title=title)

### Загрузка облаков

Получаем массивы имен облаков

In [18]:
import os

# Папка с датасетом
root_dir = "/content/pcd"

# Список пар: путь к архиву, имя архива
pcd_files = {}
for file in os.listdir(root_dir):
  pcd_files[file] = []
  sub_dir = os.path.join(root_dir, file)
  for subfile in os.listdir(sub_dir):
    file_dict = {}
    file_dict["path"] = sub_dir
    file_dict["name"] = subfile
    pcd_files[file].append(file_dict)

In [19]:
print(pcd_files.keys())

dict_keys(['clouds_stereo', 'clouds_tof'])


### Обработчики облаков точек и кластеров

Класс для обработки облака точек

In [85]:
class CloudProcessor:
  def __init__(self, cloud):
    self.cloud = cloud

  def downsample(self, voxel_size=0.03):
    self.cloud = self.cloud.voxel_down_sample(voxel_size=voxel_size)
    return self

  def alignPlaneWithZ(self, plane):
    # Параметры плоскости
    [a,b,c,d] = plane
    # Ось вращения для совмещения нормали к плоскости с осью z
    axis = np.array([b,a,0])
    # Угол поворота
    angle = np.arccos(c)
    # Приводим ось вращения в формат вектора Родриго
    axis = (axis / np.linalg.norm(axis)) * angle
    # Получаем матрицу поворота вокруг вектора на заданный угол
    R = self.cloud.get_rotation_matrix_from_axis_angle(axis)
    # Поворачиваем облако
    self.cloud.rotate(R, center=(0,0,0))
    # Переносим облако для совпадения плоскостей
    self.cloud.translate((0,0,d))
    return self
  
  def statisticalFiltration(self, nb_neighbors=20, std_ratio=0.01):
    self.cloud, ind_f1 = self.cloud.remove_statistical_outlier(nb_neighbors=nb_neighbors,
                                                      std_ratio=std_ratio)
    return self

  def radialFiltration(self, nb_points=15, radius=0.15):
    self.cloud, ind_f2 = self.cloud.remove_radius_outlier(nb_points=nb_points, radius=radius)
    return self
  
  def removePlane(self, distance_threshold=0.04, ransac_n=3, num_iterations=1000):
    plane_model, inliers = self.cloud.segment_plane(distance_threshold=0.04,
                                          ransac_n=3, num_iterations=1000)
    self.cutPoints(inliers, invert=True)
    return self

  def cutPoints(self, indices, invert=False):
    self.cloud = self.cloud.select_by_index(indices, invert=invert)
    return self

  # Кластеризация с использованием алгоритма DBSCAN
  def DbscanClusterization(self, eps=0.1, min_points=10, verbose=True):
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
      labels = np.array(self.cloud.cluster_dbscan(eps=eps, min_points=min_points, print_progress=verbose))

    max_label = labels.max()
    clusters = []
    for i in range(max_label + 1):
      cluster = o3d.geometry.PointCloud()
      cluster.points = o3d.utility.Vector3dVector(np.asarray(self.cloud.points)[labels == i])
      cluster.colors = o3d.utility.Vector3dVector(np.asarray(self.cloud.colors)[labels == i])
      clusters.append(cluster)

    return clusters


Класс для обработки кластеров облаков точек

In [75]:
class ClustersProcessor:
  def __init__(self, clusters):
    self.clusters = clusters

  def filterByNumPoints(self, min_points_num = 80):
    filtered_clusters = []
    for cl in self.clusters:
      if len(cl.points) > min_points_num:
        filtered_clusters.append(cl)
    self.clusters = filtered_clusters
    return self
    
  def getAABB(self, color = (1, 0, 0)):
    bb = []
    for cl in self.clusters:
      aabb = cl.get_axis_aligned_bounding_box()
      aabb.color = color
      bb.append(aabb)
    return bb

  def getOBB(self, color = (1, 0, 0)):
    bb = []
    for cl in self.clusters:
      obb = cl.get_oriented_bounding_box()
      obb.color = color
      bb.append(obb)
    return bb

  def createMeshes(self, alpha=0.2):
    meshes = []
    for cluster in self.clusters:
      mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(cluster, alpha=0.2)
      mesh.compute_vertex_normals()
      meshes.append(mesh)
    return meshes

Функции для опеределения пересечения обрамляющим параллелепипедом плоскости

In [23]:
def line_plane_intersect(pointA, pointB, plane_coeff):
  # Вычисление коэффициентов прямой
  A = pointB[0]-pointA[0]
  B = pointB[1]-pointA[1]
  C = pointB[2]-pointA[2]
  # Свободные коэффициенты
  D=(B*pointA[0])-(A*pointA[1])
  D2=(C*pointA[0])-(A*pointA[2])
  # Матрицы
  M1 = np.array([[B, -A, 0], [C, 0 ,-A], [plane_coeff[0], plane_coeff[1], plane_coeff[2]]])
  M2 = np.array([[D], [D2], [-plane_coeff[3]]])
  # Координата точки пересечения
  point_intersect = np.linalg.solve(M1, M2)
  return point_intersect

In [24]:
def euclideanDistance(pointA, pointB):
  dist = np.sqrt((pointB[0]-pointA[0])**2+(pointB[1]-pointA[1])**2+(pointB[2]-pointA[2])**2)
  return dist

In [25]:
# Определяем пересечение ограничивающего прямоугольника с плоскостью
def bbox_plane_intersect(bbox_points, bbox_center, plane_coeff):
  for i in range(len(bbox_points)):
    # Получаем первую точку
    pointA = bbox_points[i]
    # Координата точки пересечения
    x = line_plane_intersect(pointA, bbox_center, plane_coeff)
    # Вычисление расстояния от точки пересечения до центра и до края bbox'а
    delta1 = euclideanDistance(x, bbox_center)
    delta2 = euclideanDistance(pointA, bbox_center)
    if delta1 < delta2:
      return True
  return False

### Детектирование наличия объектов в области портала двери

In [84]:
index = 99
# Путь к облаку точек
path_to_pcd = joinPath(pcd_files['clouds_tof'][index])
# Открываем облако точек
pcd = o3d.io.read_point_cloud(path_to_pcd)
# Выводим количество точек в облаке
print(pcd)
# Создаем обработчик облака точек
tof_processor = CloudProcessor(tof_pcd)
# Сжимаем
tof_processor.downsample(voxel_size=0.03)
# Статистическая фильтрация
tof_processor.statisticalFiltration(nb_neighbors=20, std_ratio=0.01)
# Радиальная фильтрация
tof_processor.radialFiltration(nb_points=15, radius=0.2)
# Удаляем наибольшую плоскость
tof_processor.removePlane(distance_threshold=0.04, ransac_n=3, num_iterations=1000)
# Находим следующую наибольшую плоскость (предположительно дверь)
door_plane, inliers = tof_processor.cloud.segment_plane(distance_threshold=0.005, ransac_n=3, num_iterations=1000)
# Выделяем дверь из облака точек
door_cloud = tof_processor.cloud.select_by_index(inliers)
# Запоминаем обрамляющий прямоугольник двери
door_bb = door_cloud.get_axis_aligned_bounding_box()
# Удаляем задетектированную дверь
tof_processor.cutPoints(inliers, invert=True)
# Детектируем кластеры
tof_clusters = ClustersProcessor(tof_processor.DbscanClusterization(eps=0.1, min_points=10))
# Фильтруем кластеры по количеству точек в них
tof_clusters = tof_clusters.filterByNumPoints()
# Создаем mesh'и кластеров
clusters_meshes = tof_clusters.createMeshes()
# Создаем mesh портала двери
door_mesh = createMeshFromBB(door_bb)
# Вычисляем количество пересечений объектов с порталом двери
intersections = [mesh.is_intersecting(door_mesh) for mesh in clusters_meshes]
# Выводим массив пересечений с порталом
print(intersections)
# Отображаем кластеры и портал двери
drawGeometry(clusters_meshes + [door_mesh])

PointCloud with 232394 points.
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 12
[True, False, False, True, True, True, False]
